In [1]:
from PIL import Image 
import numpy as np
import os 

In [2]:
def load_image(filepath):
    im = Image.open(filepath)
    im = im.convert('L')
    arr = np.asarray(im)
    return arr

def load_images(directory):
    files = os.listdir(directory)
    images = []
    for filename in files:
        filepath = f"{directory}/{filename}"
        try:
            image = load_image(filepath)
            images.append(image)
        except Exception as e:
            print(e)
            print(f"--Failed to load file: {filename}")
    return images

In [3]:
yes_images_directory = '/Users/luchicila/Downloads/archive-2/yes'
no_images_directory = '/Users/luchicila/Downloads/archive-2/no'

no_images = load_images(no_images_directory)
yes_images = load_images(yes_images_directory)

In [4]:
im_dims = []
for im in no_images:
    im_dims.append(im.shape)
for im in yes_images:
    im_dims.append(im.shape)
im_dims = np.asarray(im_dims)
max_dims = np.max(im_dims, axis=0)

def resize_images(images, max_dims):
    im_size = (int(max_dims[0] * 0.3), int(max_dims[1] * 0.3))
    for i in range(len(images)):
        im = Image.fromarray(images[i])
        # im = im.resize((max_dims[1], max_dims[0]))
        im = im.resize((im_size[1], im_size[0]))
        images[i] = np.asarray(im)
    return images 

no_images = resize_images(no_images, max_dims)
yes_images = resize_images(yes_images, max_dims)

In [5]:
no_labes = np.zeros((len(no_images),))
yes_labes = np.ones((len(yes_images),))
X = np.concatenate([no_images, yes_images])
y = np.concatenate([no_labes, yes_labes])
X.shape, y.shape

((253, 428, 576), (253,))

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.shape, y_train.shape

((202, 428, 576), (202,))

In [7]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train[0], y_test[0]

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

In [15]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout

input_size = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
model = Sequential()

layer = Conv2D(64, kernel_size=3, activation='relu', input_shape=input_size)
model.add(layer)

layer = Conv2D(32, kernel_size=3, activation='relu')
model.add(layer)

layer = Dropout(0.2)
model.add(layer)

layer = Flatten()
model.add(layer)

layer = Dense(2, activation='softmax')
model.add(layer)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10)

Epoch 1/10
7/7 [==============================] - 55s 8s/step - loss: 2500.4861 - accuracy: 0.5594
Epoch 2/10
7/7 [==============================] - 52s 7s/step - loss: 29.5454 - accuracy: 0.6683
Epoch 3/10
7/7 [==============================] - 55s 8s/step - loss: 1.1755 - accuracy: 0.9208
Epoch 4/10
7/7 [==============================] - 55s 8s/step - loss: 0.4129 - accuracy: 0.9703
Epoch 5/10
7/7 [==============================] - 56s 8s/step - loss: 0.1433 - accuracy: 0.9752
Epoch 6/10
7/7 [==============================] - 55s 8s/step - loss: 0.0688 - accuracy: 0.9901
Epoch 7/10
7/7 [==============================] - 56s 8s/step - loss: 0.0506 - accuracy: 0.9950
Epoch 8/10
7/7 [==============================] - 56s 8s/step - loss: 0.0378 - accuracy: 0.9901
Epoch 9/10
7/7 [==============================] - 57s 8s/step - loss: 0.0273 - accuracy: 0.9802
Epoch 10/10
7/7 [==============================] - 55s 8s/step - loss: 0.0238 - accuracy: 0.9901


In [16]:
predictions = model.predict(X_test)
y_test_pred = []
for i in range(len(predictions)):
    pred_class = np.argmax(predictions[i])
    y_test_pred.append(pred_class)

y_test_class = []
for i in range(len(y_test)):
    y_class = np.argmax(y_test[i])
    y_test_class.append(y_class)

In [17]:
from sklearn.metrics import plot_confusion_matrix, accuracy_score

acc = accuracy_score(y_true=y_test_class, y_pred=y_test_pred)
print(f"Test accuracy: {acc}")

Test accuracy: 0.6666666666666666


In [18]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test_class, y_test_pred)

array([[10,  5],
       [12, 24]])

In [20]:
12 / len(y_test_class)

0.23529411764705882